In [ ]:
roc_auc: 0.9997
accuracy: 0.9817

Test Metrics:
precision_macro: 0.7433
precision_micro: 0.7405
recall_macro: 0.7277
recall_micro: 0.7255
f1_macro: 0.7327
f1_micro: 0.7329
roc_auc: 0.9542
accuracy: 0.7255

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# First, let's prepare our data
# Get feature columns (excluding the Race_ columns)
race_columns = [col for col in train_df.columns if col.startswith('Race_')]
feature_columns = [col for col in train_df.columns if not col.startswith('Race_')]

# Handle missing values in Age column
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

# Prepare X and y
X = train_df[feature_columns].values
y = train_df[race_columns].values

# Initialize scaler
scaler = StandardScaler()

def create_model(input_dim, l2_lambda=0.0001):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(len(race_columns), activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# [Rest of the code remains exactly the same]
def plot_confusion_matrices(y_true, y_pred, class_names):
    """Plot confusion matrices for each class"""
    fig, axes = plt.subplots(2, (len(class_names) + 1) // 2, figsize=(15, 8))
    axes = axes.ravel()
    
    for idx, class_name in enumerate(class_names):
        cm = confusion_matrix(y_true[:, idx], (y_pred[:, idx] > 0.5).astype(int))
        sns.heatmap(cm, annot=True, fmt='d', ax=axes[idx])
        axes[idx].set_title(f'Confusion Matrix - {class_name}')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
    
    plt.tight_layout()
    return fig

def calculate_metrics(y_true, y_pred_proba):
    """Calculate comprehensive metrics for multi-label classification"""
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'roc_auc': roc_auc_score(y_true, y_pred_proba, average='macro'),
        'accuracy': (y_true == y_pred).all(axis=1).mean()
    }
    
    # Per-class metrics
    class_metrics = {
        'precision_per_class': precision_score(y_true, y_pred, average=None),
        'recall_per_class': recall_score(y_true, y_pred, average=None),
        'f1_per_class': f1_score(y_true, y_pred, average=None),
        'roc_auc_per_class': roc_auc_score(y_true, y_pred_proba, average=None)
    }
    
    return metrics, class_metrics

# Perform 10-fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_metrics = []
all_class_metrics = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/10")
    
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create and train model
    model = create_model(X_train.shape[1])
    
    # Train the model
    history = model.fit(
        X_train_scaled, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        verbose=2
    )
    
    # Get predictions
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Calculate metrics
    train_metrics, train_class_metrics = calculate_metrics(y_train, y_pred_train)
    test_metrics, test_class_metrics = calculate_metrics(y_test, y_pred_test)
    
    # Store metrics
    all_metrics.append({
        'fold': fold + 1,
        'train': train_metrics,
        'test': test_metrics
    })
    
    all_class_metrics.append({
        'fold': fold + 1,
        'train': train_class_metrics,
        'test': test_class_metrics
    })
    
    # Print current fold metrics
    print("\nTrain Metrics:")
    for metric_name, value in train_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    print("\nTest Metrics:")
    for metric_name, value in test_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    # Plot confusion matrices for this fold
    plot_confusion_matrices(y_test, y_pred_test, race_columns)
    plt.show()

# Calculate and print final average results
print("\nFinal Average Results:")
print("\nOverall Metrics:")
for metric_name in all_metrics[0]['test'].keys():
    test_values = [fold['test'][metric_name] for fold in all_metrics]
    mean_value = np.mean(test_values)
    std_value = np.std(test_values)
    print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

print("\nPer-Class Metrics:")
for class_idx, class_name in enumerate(race_columns):
    print(f"\n{class_name}:")
    for metric_name in ['precision_per_class', 'recall_per_class', 'f1_per_class', 'roc_auc_per_class']:
        test_values = [fold['test'][metric_name][class_idx] for fold in all_class_metrics]
        mean_value = np.mean(test_values)
        std_value = np.std(test_values)
        print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

# Plot learning curves from the last fold
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os


# First, let's prepare our data
# Get feature columns (excluding the Race_ columns)
race_columns = [col for col in train_df.columns if col.startswith('Race_')]
feature_columns = [col for col in train_df.columns if not col.startswith('Race_')]

# Handle missing values in Age column
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

# Prepare X and y
X = train_df[feature_columns].values
y = train_df[race_columns].values

# Initialize scaler
scaler = StandardScaler()

def create_model(input_dim, l2_lambda=0.0001):
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim,
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(64, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(64, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(len(race_columns), activation='softmax',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda))
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# [Rest of the code remains exactly the same]
def plot_confusion_matrices(y_true, y_pred, class_names):
    """Plot confusion matrices for each class"""
    fig, axes = plt.subplots(2, (len(class_names) + 1) // 2, figsize=(15, 8))
    axes = axes.ravel()
    
    for idx, class_name in enumerate(class_names):
        cm = confusion_matrix(y_true[:, idx], (y_pred[:, idx] > 0.5).astype(int))
        sns.heatmap(cm, annot=True, fmt='d', ax=axes[idx])
        axes[idx].set_title(f'Confusion Matrix - {class_name}')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
    
    plt.tight_layout()
    return fig

def calculate_metrics(y_true, y_pred_proba):
    """Calculate comprehensive metrics for multi-label classification"""
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'roc_auc': roc_auc_score(y_true, y_pred_proba, average='macro'),
        'accuracy': (y_true == y_pred).all(axis=1).mean()
    }
    
    # Per-class metrics
    class_metrics = {
        'precision_per_class': precision_score(y_true, y_pred, average=None),
        'recall_per_class': recall_score(y_true, y_pred, average=None),
        'f1_per_class': f1_score(y_true, y_pred, average=None),
        'roc_auc_per_class': roc_auc_score(y_true, y_pred_proba, average=None)
    }
    
    return metrics, class_metrics

# Perform 10-fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_metrics = []
all_class_metrics = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/10")
    
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create and train model
    model = create_model(X_train.shape[1])
    
    # Define callbacks    
    model_checkpoint = ModelCheckpoint(
        filepath=f'model_checkpoints/model_fold_{fold+1}.keras',  # Changed from .h5 to .keras
        monitor='val_accuracy',
        save_best_only=True,     # Only save when the monitored metric is improved
        mode='max',              # We want to minimize the validation loss
        verbose=1
    )
    
    # Train the model with callbacks
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,              # Maximum number of epochs
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        callbacks=[model_checkpoint],
        verbose=2
    )
    
    # Load the best model for this fold
    best_model = load_model(f'model_checkpoints/model_fold_{fold+1}.keras')
    
    # Get predictions using the best model
    y_pred_train = best_model.predict(X_train_scaled)
    y_pred_test = best_model.predict(X_test_scaled)
    
    # Calculate metrics
    train_metrics, train_class_metrics = calculate_metrics(y_train, y_pred_train)
    test_metrics, test_class_metrics = calculate_metrics(y_test, y_pred_test)
    
    
    # Store metrics
    all_metrics.append({
        'fold': fold + 1,
        'train': train_metrics,
        'test': test_metrics
    })
    
    all_class_metrics.append({
        'fold': fold + 1,
        'train': train_class_metrics,
        'test': test_class_metrics
    })
    
    # Print current fold metrics
    print("\nTrain Metrics:")
    for metric_name, value in train_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    print("\nTest Metrics:")
    for metric_name, value in test_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    # Plot confusion matrices for this fold
    plot_confusion_matrices(y_test, y_pred_test, race_columns)
    plt.show()
    
    # Plot learning curves
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Accuracy Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Calculate and print final average results
print("\nFinal Average Results:")
print("\nOverall Metrics:")
for metric_name in all_metrics[0]['test'].keys():
    test_values = [fold['test'][metric_name] for fold in all_metrics]
    mean_value = np.mean(test_values)
    std_value = np.std(test_values)
    print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

print("\nPer-Class Metrics:")
for class_idx, class_name in enumerate(race_columns):
    print(f"\n{class_name}:")
    for metric_name in ['precision_per_class', 'recall_per_class', 'f1_per_class', 'roc_auc_per_class']:
        test_values = [fold['test'][metric_name][class_idx] for fold in all_class_metrics]
        mean_value = np.mean(test_values)
        std_value = np.std(test_values)
        print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

# Plot learning curves from the last fold
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os


# First, let's prepare our data
# Get feature columns (excluding the Race_ columns)
race_columns = [col for col in train_df.columns if col.startswith('Race_')]
feature_columns = [col for col in train_df.columns if not col.startswith('Race_')]

# Handle missing values in Age column
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

# Prepare X and y
X = train_df[feature_columns].values
y = train_df[race_columns].values

# Initialize scaler
scaler = StandardScaler()

def create_model(input_dim, l2_lambda=0.0001):
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim,
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dense(len(race_columns), activation='softmax',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda))
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# [Rest of the code remains exactly the same]
def plot_confusion_matrices(y_true, y_pred, class_names):
    """Plot confusion matrices for each class"""
    fig, axes = plt.subplots(2, (len(class_names) + 1) // 2, figsize=(15, 8))
    axes = axes.ravel()
    
    for idx, class_name in enumerate(class_names):
        cm = confusion_matrix(y_true[:, idx], (y_pred[:, idx] > 0.5).astype(int))
        sns.heatmap(cm, annot=True, fmt='d', ax=axes[idx])
        axes[idx].set_title(f'Confusion Matrix - {class_name}')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
    
    plt.tight_layout()
    return fig

def calculate_metrics(y_true, y_pred_proba):
    """Calculate comprehensive metrics for multi-label classification"""
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'roc_auc': roc_auc_score(y_true, y_pred_proba, average='macro'),
        'accuracy': (y_true == y_pred).all(axis=1).mean()
    }
    
    # Per-class metrics
    class_metrics = {
        'precision_per_class': precision_score(y_true, y_pred, average=None),
        'recall_per_class': recall_score(y_true, y_pred, average=None),
        'f1_per_class': f1_score(y_true, y_pred, average=None),
        'roc_auc_per_class': roc_auc_score(y_true, y_pred_proba, average=None)
    }
    
    return metrics, class_metrics

# Perform 10-fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_metrics = []
all_class_metrics = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/10")
    
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create and train model
    model = create_model(X_train.shape[1])
    
    # Define callbacks    
    model_checkpoint = ModelCheckpoint(
        filepath=f'model_checkpoints/model_fold_{fold+1}.keras',  # Changed from .h5 to .keras
        monitor='val_accuracy',
        save_best_only=True,     # Only save when the monitored metric is improved
        mode='max',              # We want to minimize the validation loss
        verbose=1
    )
    
    # Train the model with callbacks
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,              # Maximum number of epochs
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        callbacks=[model_checkpoint],
        verbose=2
    )
    
    # Load the best model for this fold
    best_model = load_model(f'model_checkpoints/model_fold_{fold+1}.keras')
    
    # Get predictions using the best model
    y_pred_train = best_model.predict(X_train_scaled)
    y_pred_test = best_model.predict(X_test_scaled)
    
    # Calculate metrics
    train_metrics, train_class_metrics = calculate_metrics(y_train, y_pred_train)
    test_metrics, test_class_metrics = calculate_metrics(y_test, y_pred_test)
    
    
    # Store metrics
    all_metrics.append({
        'fold': fold + 1,
        'train': train_metrics,
        'test': test_metrics
    })
    
    all_class_metrics.append({
        'fold': fold + 1,
        'train': train_class_metrics,
        'test': test_class_metrics
    })
    
    # Print current fold metrics
    print("\nTrain Metrics:")
    for metric_name, value in train_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    print("\nTest Metrics:")
    for metric_name, value in test_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    # Plot confusion matrices for this fold
    plot_confusion_matrices(y_test, y_pred_test, race_columns)
    plt.show()
    
    # Plot learning curves
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Accuracy Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Calculate and print final average results
print("\nFinal Average Results:")
print("\nOverall Metrics:")
for metric_name in all_metrics[0]['test'].keys():
    test_values = [fold['test'][metric_name] for fold in all_metrics]
    mean_value = np.mean(test_values)
    std_value = np.std(test_values)
    print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

print("\nPer-Class Metrics:")
for class_idx, class_name in enumerate(race_columns):
    print(f"\n{class_name}:")
    for metric_name in ['precision_per_class', 'recall_per_class', 'f1_per_class', 'roc_auc_per_class']:
        test_values = [fold['test'][metric_name][class_idx] for fold in all_class_metrics]
        mean_value = np.mean(test_values)
        std_value = np.std(test_values)
        print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

# Plot learning curves from the last fold
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os


# First, let's prepare our data
# Get feature columns (excluding the Race_ columns)
race_columns = [col for col in train_df.columns if col.startswith('Race_')]
feature_columns = [col for col in train_df.columns if not col.startswith('Race_')]

# Handle missing values in Age column
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

# Prepare X and y
X = train_df[feature_columns].values
y = train_df[race_columns].values

# Initialize scaler
scaler = StandardScaler()

def create_model(input_dim, l2_lambda=0.0001):
    initial_learning_rate = 0.001
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim,
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dropout(0.05),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dropout(0.05),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dropout(0.05),
        Dense(128, activation='relu',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda)),
        Dropout(0.05),
        Dense(len(race_columns), activation='softmax',
              kernel_regularizer=l2(l2_lambda),
              bias_regularizer=l2(l2_lambda))
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# [Rest of the code remains exactly the same]
def plot_confusion_matrices(y_true, y_pred, class_names):
    """Plot confusion matrices for each class"""
    fig, axes = plt.subplots(2, (len(class_names) + 1) // 2, figsize=(15, 8))
    axes = axes.ravel()
    
    for idx, class_name in enumerate(class_names):
        cm = confusion_matrix(y_true[:, idx], (y_pred[:, idx] > 0.5).astype(int))
        sns.heatmap(cm, annot=True, fmt='d', ax=axes[idx])
        axes[idx].set_title(f'Confusion Matrix - {class_name}')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
    
    plt.tight_layout()
    return fig

def calculate_metrics(y_true, y_pred_proba):
    """Calculate comprehensive metrics for multi-label classification"""
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'precision_micro': precision_score(y_true, y_pred, average='micro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'recall_micro': recall_score(y_true, y_pred, average='micro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'roc_auc': roc_auc_score(y_true, y_pred_proba, average='macro'),
        'accuracy': (y_true == y_pred).all(axis=1).mean()
    }
    
    # Per-class metrics
    class_metrics = {
        'precision_per_class': precision_score(y_true, y_pred, average=None),
        'recall_per_class': recall_score(y_true, y_pred, average=None),
        'f1_per_class': f1_score(y_true, y_pred, average=None),
        'roc_auc_per_class': roc_auc_score(y_true, y_pred_proba, average=None)
    }
    
    return metrics, class_metrics

# Perform 10-fold cross validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
all_metrics = []
all_class_metrics = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\nFold {fold + 1}/10")
    
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create and train model
    model = create_model(X_train.shape[1])
    
    # Define callbacks    
    model_checkpoint = ModelCheckpoint(
        filepath=f'model_checkpoints/model_fold_{fold+1}.keras',  # Changed from .h5 to .keras
        monitor='val_accuracy',
        save_best_only=True,     # Only save when the monitored metric is improved
        mode='max',              # We want to minimize the validation loss
        verbose=1
    )
    
    # Train the model with callbacks
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,              # Maximum number of epochs
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        callbacks=[model_checkpoint],
        verbose=2
    )
    
    # Load the best model for this fold
    best_model = load_model(f'model_checkpoints/model_fold_{fold+1}.keras')
    
    # Get predictions using the best model
    y_pred_train = best_model.predict(X_train_scaled)
    y_pred_test = best_model.predict(X_test_scaled)
    
    # Calculate metrics
    train_metrics, train_class_metrics = calculate_metrics(y_train, y_pred_train)
    test_metrics, test_class_metrics = calculate_metrics(y_test, y_pred_test)
    
    
    # Store metrics
    all_metrics.append({
        'fold': fold + 1,
        'train': train_metrics,
        'test': test_metrics
    })
    
    all_class_metrics.append({
        'fold': fold + 1,
        'train': train_class_metrics,
        'test': test_class_metrics
    })
    
    # Print current fold metrics
    print("\nTrain Metrics:")
    for metric_name, value in train_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    print("\nTest Metrics:")
    for metric_name, value in test_metrics.items():
        print(f"{metric_name}: {value:.4f}")
    
    # Plot confusion matrices for this fold
    plot_confusion_matrices(y_test, y_pred_test, race_columns)
    plt.show()
    
    # Plot learning curves
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.axvline(x=np.argmin(history.history['val_loss']), color='r', linestyle='--', 
                label='Best Model')
    plt.title('Model Accuracy Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Calculate and print final average results
print("\nFinal Average Results:")
print("\nOverall Metrics:")
for metric_name in all_metrics[0]['test'].keys():
    test_values = [fold['test'][metric_name] for fold in all_metrics]
    mean_value = np.mean(test_values)
    std_value = np.std(test_values)
    print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

print("\nPer-Class Metrics:")
for class_idx, class_name in enumerate(race_columns):
    print(f"\n{class_name}:")
    for metric_name in ['precision_per_class', 'recall_per_class', 'f1_per_class', 'roc_auc_per_class']:
        test_values = [fold['test'][metric_name][class_idx] for fold in all_class_metrics]
        mean_value = np.mean(test_values)
        std_value = np.std(test_values)
        print(f"{metric_name}: {mean_value:.4f} ± {std_value:.4f}")

# Plot learning curves from the last fold
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()